## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [15]:
df = pd.read_csv('redfin_2023-05-05-11-18-07.csv')

## Data Clean

In [16]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [17]:
df = df.dropna(subset=['SOLD DATE'])

In [18]:
# Define list of desired months (excluding current month)
desired_months = ['April']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [19]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

False    851
Name: PRICE, dtype: int64
-------
False    851
Name: $/SQUARE FEET, dtype: int64
-------
False    851
Name: YEAR BUILT, dtype: int64
-------


In [20]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [21]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [25]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
378,PAST SALE,April-24-2023,Condo/Co-op,300 S Australian Way #1117,West Palm Beach,FL,33401.0,32200.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/300-...,Beaches MLS,RX-10871651,N,Y,26.711734,-80.064013
122,PAST SALE,April-5-2023,Condo/Co-op,1300 S Broadway St #30,Lantana,FL,33462.0,58000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lantana/1300-S-Broad...,Beaches MLS,RX-10870523,N,Y,26.573514,-80.057116
260,PAST SALE,April-7-2023,Condo/Co-op,110 Lake Meryl Dr #213,West Palm Beach,FL,33411.0,70000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/110-...,Beaches MLS,RX-10859414,N,Y,26.697770,-80.162683
41,PAST SALE,April-18-2023,Condo/Co-op,462 Monaco J #462,Delray Beach,FL,33446.0,75000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/462-Mon...,Beaches MLS,RX-10864709,N,Y,26.442239,-80.151902
582,PAST SALE,April-27-2023,Condo/Co-op,3120 Cynthia Ln #207,Lake Worth Beach,FL,33461.0,75000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth-Beach/312...,Beaches MLS,RX-10863597,N,Y,26.603799,-80.075806
676,PAST SALE,April-28-2023,Condo/Co-op,54 Dorchester C,West Palm Beach,FL,33417.0,79500.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/54-D...,Beaches MLS,RX-10855214,N,Y,26.720047,-80.129339
698,PAST SALE,April-3-2023,Condo/Co-op,210 Dorchester I,West Palm Beach,FL,33417.0,80000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/210-...,Beaches MLS,RX-10841451,N,Y,26.720629,-80.128772
169,PAST SALE,April-6-2023,Condo/Co-op,814 W Tiffany Dr #4,Mangonia Park,FL,33407.0,80000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Mangonia-Park/814-W-...,Beaches MLS,RX-10874030,N,Y,26.756847,-80.084013
836,PAST SALE,April-10-2023,Condo/Co-op,63 Windsor C,West Palm Beach,FL,33417.0,85000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/63-W...,Beaches MLS,RX-10815881,N,Y,26.711075,-80.136530
177,PAST SALE,April-26-2023,Condo/Co-op,595 Brittany M #595,Delray Beach,FL,33446.0,86000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/595-Bri...,MARMLS,A11312694,N,Y,26.444317,-80.153314


In [26]:
print(df_filtered['URL'].iloc[260])

https://www.redfin.com/FL/West-Palm-Beach/110-Lake-Meryl-Dr-33411/unit-213/home/42163000


In [24]:
# Correct the prices, if needed
df_filtered.at[827,'PRICE']=(110000)
df_filtered.at[833,'PRICE']=(234000)

In [27]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [31]:
# # Corrections, if needed
df_filtered.at[833,'$/SQUARE FEET']=(234000/1054)
df_filtered.at[827,'$/SQUARE FEET']=(110000/680)

In [32]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
378,32200.0,300 S Australian Way #1117,West Palm Beach,48.0
399,118500.0,3 Westgate Ln Unit A,Boynton Beach,69.0
795,90000.0,1653 Bridgewood Dr #1653,Boca Raton,86.0
260,70000.0,110 Lake Meryl Dr #213,West Palm Beach,88.0
138,160000.0,30 Westgate Ln Unit A,Boynton Beach,88.0
296,95000.0,6845 Willow Wood Dr #3086,Boca Raton,90.0
122,58000.0,1300 S Broadway St #30,Lantana,94.0
169,80000.0,814 W Tiffany Dr #4,Mangonia Park,95.0
486,175000.0,10173 Andover Coach Cir Unit G2,Wellington,97.0
358,130000.0,1307 SE High Point Way SE Unit C,Delray Beach,102.0


In [30]:
print(df_filtered.URL.iloc[827])

https://www.redfin.com/FL/West-Palm-Beach/133-Southampton-B-33417/home/42150562


In [17]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [33]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [34]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [35]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [36]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [37]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [38]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"April 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [39]:
m.save('index.html')

## Summary Info

In [40]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [41]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [42]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [81]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami/303-NE-187th-St-33179/unit-725/home/43010542


In [43]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
259 Worth Ave Condo, 415 Hibiscus Ave #0 Palm Beach | Price $18,000,000 | $1,823 psf | Year built: 2021
Least Expensive
Edge Condo One, 300 S Australian Way #1117 West Palm Beach | Price $32,200 | $48 psf | Year built: 2007
Highest Price Per Square Foot
2100 Sloan's Curve, 2100 S Ocean Blvd Unit 308n Palm Beach | Price $8,950,000 | $2,805 psf | Year built: 1981
Lowest Price Per Square Foot
Edge Condo One, 300 S Australian Way #1117 West Palm Beach | Price $32,200 | $48 psf | Year built: 2007
Newest
259 Worth Ave Condo, 415 Hibiscus Ave #0 Palm Beach | Price $18,000,000 | $1,823 psf | Year built: 2021
Oldest
GULFSTREAM CONDO, 31 S Golfview Rd #2 Lake Worth | Price $160,000 | $427 psf | Year built: 1948


## Time on Market Calculator

In [29]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [82]:
from datetime import datetime, timedelta

date1 = datetime(2022, 2, 23) ## List (Earlier) date
date2 = datetime(2023, 4, 11) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

412


## Map URL Snagger

In [44]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [45]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PB_condo_sales_month_ending_april_2023


## Get Summary Data

In [46]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 851
--------
Total sale price: $432,062,491
--------
Median sale price: $268,000
--------
Max sale price: $18,000,000
--------
Min sale price: $32,200
------------------------------------------------
PSF INFO
Max price per square foot: $2,805
--------
Min price per square foot: $48
--------
Median price per square foot: $233
------------------------------------------------
CONDO AGES
Newest building: 2021.0
----------
Oldest building: 1948.0
----------
Average building age: 1982.581668625147
